# Forecasting Average Rice Price in Indonesia with SARIMA

# Input Data
Data  : Rata-Rata Harga Beras Premium Indonesia 2013-2024 <br>
Sumber: Badan Pusat Statistik

In [ ]:
# INPUT DATA
harga_beras <- read.csv2("../input/harga-beras-premium-fix/harga-beras-premium-02-24.csv")
head(harga_beras)

Terlihat bahwasanya data masih sangat belum rapih, maka perlu dilakukan data cleaning terlebih dahulu

# Cleaning Data

In [ ]:
# CLEANING DATA
beras_clean <- harga_beras[-1,]
beras_clean <- as.data.frame(t(as.matrix(beras_clean)))

dim(beras_clean)

In [ ]:
colnames(beras_clean) <- c("Bulan","Harga_Beras")
beras_clean <- beras_clean[-1,]
rownames(beras_clean) <- NULL

beras_clean

Terdapat data rata-rata tahunan, karena tidak diperlukan maka data-data tersebut harus dihapuskan.

In [ ]:
# Mengubah kolom Harga_Beras menjadi numerik
beras_clean$Harga_Beras <- as.numeric(as.character(beras_clean$Harga_Beras))

# Membuat time series
harga_tseries <- ts(beras_clean$Harga_Beras, frequency = 12, start = c(2013, 1))

# Cek hasil
harga_tseries

# Exploratory Data Analysis

In [ ]:
# EDA
plot(harga_tseries,col="darkblue",ylab="Harga Beras",
     main="Rata-rata Harga Beras Premium Indonesia")
boxplot(split(harga_tseries,cycle(harga_tseries)),names=month.abb,
        main="Rata-Rata Harga Beras Premium (Per Bulan)")

In [ ]:
# Menghitung moving average dengan jendela lebar yang berbeda
ma3 <- stats::filter(harga_tseries, rep(1/2, 2), sides = 2)
ma6 <- stats::filter(harga_tseries, rep(1/4, 4), sides = 2)
ma12 <- stats::filter(harga_tseries, rep(1/9, 9), sides = 2)

# Plot deret waktu harga_tseries
plot(harga_tseries, col = "darkblue", ylab = "Harga Beras",
     main = "Rata-rata Harga Beras Premium Indonesia")

# Tambahkan garis moving average dengan jendela lebar yang berbeda
lines(ma3, col = "red", lty = 1)
lines(ma6, col = "green", lty = 2)
lines(ma12, col = "blue", lty = 3)

# Tambahkan legenda
legend("topleft", legend = c("Data", "Moving Average (Window Size = 2 bulan)", "Moving Average (Window Size = 4 bulan)", "Moving Average (Window Size = 9 bulan)"),
       col = c("darkblue", "red", "green", "blue"), lty = c(1, 1, 2, 3), cex = 0.8)


In [ ]:
# Hitung regresi linear
linear_reg <- lm(harga_tseries ~ time(harga_tseries))

# Tampilkan summary regresi linear
summary_linear_reg <- summary(linear_reg)

# Tampilkan persamaan regresi linear
cat("Persamaan Regresi Linear:\n")
cat("Harga_Beras =", round(summary_linear_reg$coefficients[1, 1], 4), "+", round(summary_linear_reg$coefficients[2, 1], 4), "* Waktu\n")

# Visualisasi regresi linear dengan scatter plot
plot(harga_tseries, col = "darkblue", ylab = "Harga Beras",
     main = "Regresi Linear: Rata-rata Harga Beras Premium Indonesia")

# Tambahkan garis regresi linear (tanpa menggunakan type = "l" untuk menggambar garis)
abline(linear_reg, col = "green")

# Tambahkan legenda
legend("topleft", legend = "Regresi Linear", col = "green", lty = 1)

# Menghitung korelasi antara waktu dan harga beras
correlation <- cor(time(harga_tseries), harga_tseries)

# Menampilkan nilai korelasi
cat("Korelasi antara Waktu dan Harga Beras:", round(correlation, 4), "\n")

Terlihat pada Time Series Plot data memiliki trend naik dan diduga memiliki pola musiman. Dari hasil boxplot dapat diambil informasi bahwa tidak ada data pencilan untuk setiap bulannya.

# Fit ARIMA Model

In [ ]:
# TRAIN-TEST SPLIT
round(0.8*length(harga_tseries))
train_data <- ts(harga_tseries[1:134],frequency=12,start=c(2013,1))
test_data <- ts(harga_tseries[134:136],frequency=12,start=c(2024,3))

In [ ]:
library(forecast)

In [ ]:
model <- auto.arima(train_data)
model
forecast_harga <- forecast(model,h=length(test_data)+6)

Didapat model SARIMA(1,1,0)(2,0,0)12 merupakan model terbaik.

# Forecasting

In [ ]:
# PLOT
plot(forecast_harga)
lines(test_data,col="red")
legend("topleft",lty=1,bty = "n",col=c("red","blue"),c("testData","ARIMAPred"))

Forecast untuk 6 bulan selanjutnya.